# The Knight's Tour

This notebook computes a solution to the [knight's tour](https://en.wikipedia.org/wiki/Knight%27s_tour) using the constraint solver `Z3`.  

In [1]:
import z3

Given an integer from the set $\{0, 1, \cdots, 63\}$, the function `row(i)` computes the name of the variable that specifies the *row* of the knight after its $i^{\textrm{th}}$ move.

In [2]:
def row(i):
    return f'R{i}'

Given an integer from the set $\{0, 1, \cdots, 63\}$, the function `col(i)` computes the name of the variable that specifies the *column* of the knight after its $i^{\textrm{th}}$ move.

In [3]:
def col(i):
    return f'C{i}'

The function `is_knight_move(row, col, rowX, colX)` takes three arguments:
* `row` is a `Z3` variable that specifies the row of the position of the knight before the move.
* `col` is a `Z3` variable that specifies the column of the position of the knight before the move.
* `rowX` is a `Z3` variable that specifies the row of the position of the knight after the move.
* `colX` is a `Z3` variable that specifies the column of the position of the knight after the move.

It returns a formula that specifies that the specified position represents a legal move for a knight.

In order to form the *conjunction* of two formulas we use the function `z3.And`, 
while the *disjunction* is build with the function `z3.Or`.  Note that these functions can be called with 
any number of arguments.

The figure below shows the moves of a knight:  The knight on `e4` can jump to all red squares.
<img src="knight-moves.png" width="50%">

In [4]:
S = {1, 2, -1, -2}
{(x, y) for x in S for y in S if abs(x) != abs(y)}

{(-2, -1), (-2, 1), (-1, -2), (-1, 2), (1, -2), (1, 2), (2, -1), (2, 1)}

In [5]:
def is_knight_move(row, col, rowX, colX):
    Formulas = set()
    S = {1, 2, -1, -2}
    DeltaSet = {(x, y) for x in S for y in S if abs(x) != abs(y)}
    for delta_r, delta_c in DeltaSet:
        Formulas.add(z3.And(rowX == row + delta_r, colX == col + delta_c))
    return z3.Or(Formulas)

The function `all_different` takes two arguments:
* `Rows` is a list of `Z3` variables. The variable `Rows[i]` specifies the row of the position of the knight after the $i^{\textrm{th}}$ move.
* `Cols` is a list of `Z3` variables. The variable `Cols[i]` specifies the column of the position of the knight after the $i^{\textrm{th}}$ move.

The function returns a set of formulas stating that for $i \not= j$ the positions after the $i^{\textrm{th}}$ move
differs from the position after the $j^{\textrm{th}}$ move.

In [6]:
def all_different(Rows, Cols):
    Result = set()
    for i in range(62+1):
        for j in range (i+1, 63+1):
            Result.add(z3.Or(Rows[i] != Rows[j], Cols[i] != Cols[j]))
    return Result

The function `all_constraints` takes two arguments:
* `Rows` is a list of `Z3` variables. The variable `Rows[i]` specifies the row of the position of the knight after the $i^{\textrm{th}}$ move.
* `Cols` is a list of `Z3` variables. The variable `Cols[i]` specifies the column of the position of the knight after the $i^{\textrm{th}}$ move.

`all_constraints` returns a set containing all constraints of the problem.

In [7]:
def all_constraints(Rows, Cols):
    Constraints = all_different(Rows, Cols)
    Constraints.add(Rows[0] == 0)
    Constraints.add(Cols[0] == 0)
    for i in range(62+1):
        Constraints.add(is_knight_move(Rows[i], Cols[i], Rows[i+1], Cols[i+1]))
    for i in range(63+1):
        Constraints.add(Rows[i] >= 0) 
        Constraints.add(Cols[i] >= 0) 
    return Constraints

The function `solve()` computes a solution of the knight's problem and returns this solution.

In [8]:
def solve():
    Rows = [z3.BitVec(row(i), 4) for i in range(63+1)]
    Cols = [z3.BitVec(col(i), 4) for i in range(63+1)]
    Constraints = all_constraints(Rows, Cols)
    S = z3.Solver()
    S.add(Constraints)
    result = str(S.check())
    if result == 'sat':
        Model    = S.model()
        Solution = (  { row(i): Model[Rows[i]] for i in range(63+1) } 
                    | { col(i): Model[Cols[i]] for i in range(63+1) })
        return Solution
    elif result == 'unsat':
        print('The problem is not solvable.')
    else:
        print('Z3 cannot determine whether the problem is solvable.')

Unfortunately, the execution time of the following cell varies greatly between
different runs.  Sometimes the cell runs in less one minute and 28 seconds, sometimes 
it might take 30 minutes.

In [9]:
%%time
Solution = solve()
Solution

CPU times: user 2min 15s, sys: 65.1 ms, total: 2min 15s
Wall time: 2min 17s


{'R0': 0,
 'R1': 2,
 'R2': 4,
 'R3': 6,
 'R4': 5,
 'R5': 7,
 'R6': 6,
 'R7': 4,
 'R8': 3,
 'R9': 1,
 'R10': 2,
 'R11': 0,
 'R12': 2,
 'R13': 3,
 'R14': 4,
 'R15': 2,
 'R16': 0,
 'R17': 1,
 'R18': 3,
 'R19': 1,
 'R20': 0,
 'R21': 1,
 'R22': 3,
 'R23': 2,
 'R24': 0,
 'R25': 2,
 'R26': 1,
 'R27': 0,
 'R28': 2,
 'R29': 3,
 'R30': 1,
 'R31': 0,
 'R32': 1,
 'R33': 3,
 'R34': 5,
 'R35': 4,
 'R36': 2,
 'R37': 0,
 'R38': 1,
 'R39': 3,
 'R40': 4,
 'R41': 5,
 'R42': 6,
 'R43': 7,
 'R44': 5,
 'R45': 7,
 'R46': 6,
 'R47': 4,
 'R48': 3,
 'R49': 5,
 'R50': 7,
 'R51': 6,
 'R52': 7,
 'R53': 6,
 'R54': 4,
 'R55': 5,
 'R56': 7,
 'R57': 6,
 'R58': 7,
 'R59': 5,
 'R60': 6,
 'R61': 4,
 'R62': 5,
 'R63': 7,
 'C0': 0,
 'C1': 1,
 'C2': 0,
 'C3': 1,
 'C4': 3,
 'C5': 4,
 'C6': 6,
 'C7': 7,
 'C8': 5,
 'C9': 4,
 'C10': 2,
 'C11': 1,
 'C12': 0,
 'C13': 2,
 'C14': 4,
 'C15': 3,
 'C16': 2,
 'C17': 0,
 'C18': 1,
 'C19': 2,
 'C20': 4,
 'C21': 6,
 'C22': 7,
 'C23': 5,
 'C24': 6,
 'C25': 7,
 'C26': 5,
 'C27': 7,
 'C28': 

The function `create_board(Solution)` returns a matrix `Board` of size $8\times 8$.
The following holds:
$$ \texttt{Board}[\texttt{R}i][\texttt{C}i] = i $$
Therefore, if `Board[r][c] == i`, then at the beginning of the $i^{\textrm{th}}$ move the knight is located in row `r` and column `c`. 

In [10]:
def create_board(Solution):
    Board = [[0 for _ in range(8)] for _ in range(8)]
    for i in range(1, 63+1):
        r = Solution[row(i)].as_long()
        c = Solution[col(i)].as_long()
        Board[r][c] = i
    return Board

In [11]:
create_board(Solution)

[[0, 11, 16, 37, 20, 31, 24, 27],
 [17, 38, 19, 30, 9, 26, 21, 32],
 [12, 1, 10, 15, 36, 23, 28, 25],
 [39, 18, 13, 48, 29, 8, 33, 22],
 [2, 47, 40, 35, 14, 61, 54, 7],
 [41, 44, 49, 4, 55, 34, 59, 62],
 [46, 3, 42, 51, 60, 57, 6, 53],
 [43, 50, 45, 56, 5, 52, 63, 58]]

The function `print_board` prints the given `Board`.

In [12]:
def print_board(Board):
    n = len(Board)
    # Determine the width of the widest element in the matrix
    width = max([ len(str(element)) for row in Board
                                    for element in row
                ])
    # Create the top and bottom of the matrix
    top_line = '╔'
    for i in range(n - 1):
        top_line += '═' * (width + 2) + '╦'
    top_line += '═' * (width + 2) + '╗'
    mid_line = '╠'
    for i in range(n - 1):
        mid_line += '═' * (width + 2) + '╬'
    mid_line += '═' * (width + 2) + '╣'    
    bot_line = '╚'
    for i in range(n - 1):
        bot_line += '═' * (width + 2) + '╩'
    bot_line += '═' * (width + 2) + '╝'
    # Print the top of the matrix
    print(top_line)
    # Iterate through the rows and columns of the matrix, and print
    # each element with proper padding
    for i, row in enumerate(Board):
        line = '\u2551'
        for element in row:
            line += f' {element:>{width}} ║'
        print(line)
        # Print a horizontal line
        if i < len(Board) - 1:
            print(mid_line)
    # Print the bottom of the matrix
    print(bot_line)

In [13]:
print_board(create_board(Solution))

╔════╦════╦════╦════╦════╦════╦════╦════╗
║  0 ║ 11 ║ 16 ║ 37 ║ 20 ║ 31 ║ 24 ║ 27 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 17 ║ 38 ║ 19 ║ 30 ║  9 ║ 26 ║ 21 ║ 32 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 12 ║  1 ║ 10 ║ 15 ║ 36 ║ 23 ║ 28 ║ 25 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 39 ║ 18 ║ 13 ║ 48 ║ 29 ║  8 ║ 33 ║ 22 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║  2 ║ 47 ║ 40 ║ 35 ║ 14 ║ 61 ║ 54 ║  7 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 41 ║ 44 ║ 49 ║  4 ║ 55 ║ 34 ║ 59 ║ 62 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 46 ║  3 ║ 42 ║ 51 ║ 60 ║ 57 ║  6 ║ 53 ║
╠════╬════╬════╬════╬════╬════╬════╬════╣
║ 43 ║ 50 ║ 45 ║ 56 ║  5 ║ 52 ║ 63 ║ 58 ║
╚════╩════╩════╩════╩════╩════╩════╩════╝


# Visualization

If you have not yet installed `chess-problem-visuals` you have to uncomment the following line.

In [14]:
!pip install git+https://github.com/reclinarka/chess-problem-visuals

  Cloning https://github.com/reclinarka/chess-problem-visuals to /private/var/folders/q9/qftgdjx91wx4s5jcqkfz5bd00000gn/T/pip-req-build-raxsl4qy
  Running command git clone --filter=blob:none --quiet https://github.com/reclinarka/chess-problem-visuals /private/var/folders/q9/qftgdjx91wx4s5jcqkfz5bd00000gn/T/pip-req-build-raxsl4qy
  Resolved https://github.com/reclinarka/chess-problem-visuals to commit 764a29b376fe9dd3cbb2623ce8740f73c6711fa4
  Preparing metadata (setup.py) ... done


In [15]:
from chess_problem_visuals import problem_board

The function `show_solution` displays the given solution on a chessboard.
The solution `Board` is represented as a list of lists.  We have `Board[row][col] == k` if the $k^\textrm{th}$ move leads the knight to the position `(row, col)`.

In [16]:
def show_solution(Board, width="50%"):
    n         = len(Board)
    Positions = {}
    for row in range(n):
        for col in range(n):
            k = Board[row][col]
            Positions[k] = (col, row)
    start = (0, 0)
    Path  = [start]
    for k in range(1, n*n):
        Path.append(Positions[k])
    Visual = problem_board(n, K_start=start, K_path=Path, 
                           html_width=width, 
                           arrow_color="darkblue",
                           arrow_width=0.2)
    return Visual

In [17]:
show_solution(create_board(Solution))